# Надёжность заёмщиков

Кредитный отдел банка просит разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Для исследования предоставлена статистика о платёжеспособности клиентов.

По результатам исследования будет строиться модель кредитного скоринга, который оценит способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

Чтобы изучить данные датафрейма, сделать необходимые рассчёты, провести исследование и составить отчёт сразу импортируем библиотеки, который нам потребуются в данном проекте

In [1]:
from collections import Counter
from io import BytesIO
from pymystem3 import Mystem

import pandas as pd
import requests

Напишем функцию, чтобы прочитать и сохранить  данные из Google Sheets

In [2]:
def table(spreadsheet_id):
    spreadsheet_id = spreadsheet_id
    file_name = 'https://docs.google.com/spreadsheets/d/{}/export?format=csv'.format(spreadsheet_id)
    r = requests.get(file_name)
    table = pd.read_csv(BytesIO(r.content))
    return table

Прочитаем и сохраним файл, который предоставил заказчик

In [3]:
df = table('1pVhghFnWxLW6IZCLMms3xO7J6ErJGY2_cJosZyvs7_A')# чтение файла с данными и сохранение в df
df.head(10) # получение первых 10 строк таблицы df

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Получим общую информацию о таблице:

In [4]:
df.info() # получение общей информации о данных в таблице df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


В таблице 12 столбцов. Типы данных в столбцах — `float`, `int`, `object`.

Согласно документации к данным:
* `children` — количество детей в семье
* `days_employed` — общий трудовой стаж в днях
* `dob_years` — возраст клиента в годах
* `education` — уровень образования клиента
* `education_id` — идентификатор уровня образования
* `family_status` — семейное положение
* `family_status_id` — идентификатор семейного положения
* `gender` — пол клиента
* `income_type` — тип занятости
* `debt` — имел ли задолженность по возврату кредитов
* `total_income` — ежемесячный доход
* `purpose` — цель получения кредита


Количество значений в столбцах различается. Значит, в данных есть пропущенные значения.


**Вывод**

В каждой строке таблицы — данные о заёмщике банка. Часть колонок описывает самого заёмщика: возраст, пол, образование, семейное положение, число детей, трудовой стаж, тип должности и ежемесячном доходе. Остальные данные рассказывают о данных для получения кредита: имел ли задолжность по возврату кредитов, цель получения кредита.

Предварительно можно утверждать, что, данных достаточно для проверки гипотез. Но встречаются различные проблемы в данных.

Чтобы двигаться дальше, нужно устранить проблемы в данных.

## Шаг 2. Предобработка данных

Исправим данные и приведём их к пригодному для анализа виду.

### Обработка пропусков

Сначала посчитаем, сколько в таблице пропущенных значений:

In [5]:
df.isna().sum() # подсчёт пропусков

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

В таблице имеются пропуски в двух столбцах по ежемесячному доходу и по общему трудовому стажу. В данном случае нужно заменить пропуски в количественных переменных.

Сравним среднее и медианное значение в столбцах с пропусками, предварительно сгруппировав заёмщиков по типу занятости.

Медианное значение ежемесячного дохода заёмщиков сгруппированных по типу занятости:

In [6]:
df.groupby('income_type')['total_income'].median() # группируем по типу занятости доход и считаем медиану

income_type
безработный        131339.751676
в декрете           53829.130729
госслужащий        150447.935283
компаньон          172357.950966
пенсионер          118514.486412
предприниматель    499163.144947
сотрудник          142594.396847
студент             98201.625314
Name: total_income, dtype: float64

Среднее значение ежемесячного дохода заёмщиков сгруппированных по типу занятости:

In [7]:
df.groupby('income_type')['total_income'].mean() # группируем по типу занятости доход и считаем среднее значение

income_type
безработный        131339.751676
в декрете           53829.130729
госслужащий        170898.309923
компаньон          202417.461462
пенсионер          137127.465690
предприниматель    499163.144947
сотрудник          161380.260488
студент             98201.625314
Name: total_income, dtype: float64

Медианное значение общего трудового стажа заёмщиков сгруппированных по типу занятости:

In [8]:
df.groupby('income_type')['days_employed'].median() # группируем по типу занятости стаж и считаем медиану

income_type
безработный        366413.652744
в декрете           -3296.759962
госслужащий         -2689.368353
компаньон           -1547.382223
пенсионер          365213.306266
предприниматель      -520.848083
сотрудник           -1574.202821
студент              -578.751554
Name: days_employed, dtype: float64

Среднее значение общего трудового стажа заёмщиков сгруппированных по типу занятости:

In [9]:
df.groupby('income_type')['days_employed'].mean() # группируем по типу занятости стаж и считаем среднее значение

income_type
безработный        366413.652744
в декрете           -3296.759962
госслужащий         -3399.896902
компаньон           -2111.524398
пенсионер          365003.491245
предприниматель      -520.848083
сотрудник           -2326.499216
студент              -578.751554
Name: days_employed, dtype: float64

**Вывод** 

При сравнении среднего и медианного значение данных рассматриваемых столбцов видно, что среднее значение у работающих заёмщиков завышено по сравнению с медианным в обоих столбцах. 
Следовательно использовать медианных значения будет более корректно.

**Заменим пропущенные значения ежемесячного дохода медианным значением для каждого типа занятости:**

In [10]:
df['total_income'] = df.groupby('income_type')['total_income'].transform(lambda x: x.fillna(x.median())) # используем метод transform из библиотеки pandas
df['days_employed'] = df.groupby('income_type')['days_employed'].transform(lambda x: x.fillna(x.median())) # используем метод transform из библиотеки pandas

Убедимся, что в таблице не осталось пропусков. Для этого ещё раз посчитаем пропущенные значения:

In [11]:
df.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

**Вывод**

Причины в пропусках данных связаны с заполнением кредитной заявки и указанием дохода и/или стажа работы. Видимо, если потенциальный заёмщик не имеет дохода или не работает, то он оставляет поле пустым, а поля доход и стаж работы по всей видимости связаны, тк пропуски наблюдаются в одинаковых строчках, то есть для каждого заёмщика и число пропусков совпадает. 

Значения были успешно заменены и пропусков более не наблюдается.

Вызывают вопросы отрицательные значения в столбце общий трудовой стаж, но к этому вопросу вернёмся позднее, если эти данные будут влиять на исследование.

### Замена типа данных

Заменим вещественные данные на целочисленные

In [12]:
df['total_income'] = df['total_income'].astype('int') # заменим вещественные данные на числовые используя метод astype в доходе заёмщиков
df['days_employed'] = df['days_employed'].astype('int') # заменим вещественные данные на числовые используя метод astype в стаже заёмщиков

Убедимся, что в таблице в столбцах не осталось вещественных данных:

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21525 non-null  int64 
 1   days_employed     21525 non-null  int32 
 2   dob_years         21525 non-null  int64 
 3   education         21525 non-null  object
 4   education_id      21525 non-null  int64 
 5   family_status     21525 non-null  object
 6   family_status_id  21525 non-null  int64 
 7   gender            21525 non-null  object
 8   income_type       21525 non-null  object
 9   debt              21525 non-null  int64 
 10  total_income      21525 non-null  int32 
 11  purpose           21525 non-null  object
dtypes: int32(2), int64(5), object(5)
memory usage: 1.8+ MB


Удалим отрицательные значения из столбца стажа работы, тк для работающих заёмщиков данные по стажу указаны корректно в днях:

In [14]:
df['days_employed'] = df['days_employed'].abs() # заменим все значения в столбце 'days_employed' на их же значения по модулю используя метод abs()

Удалим отрицательные значения из столбца дети, тк отрицательного числа детей быть не может:

In [15]:
df['children'] = df['children'].abs()

**Вывод**

Вещественные данные были заменены на целочисленные.
<br>
Использовался метод `astype()`, а не `to_numeric()`, тк нужно было привести столбцы к целочисленным значениям, а метод `to_numeric()` заменяет на вещественные.
<br>
<br>
Кроме прочего, значения в столбеце со стажем работы были приведены к положительным значениям, тк для работающих заёмщиков по типу занятости данные по стажу указаны корректно в днях, а для неработающих по типу занятости эти данные использовать не имеет смысла. 


### Обработка дубликатов

Посчитаем дубликаты в таблице, чтобы в дальнейшем избавиться от них.<br>
Для этого проверим все столбцы с типом `object`.<br>
Проверяем столбец `education`:

In [16]:
df['education'].value_counts() # Просмотр уникальных значений образования и подсчёт их количества 

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

Проверяем столбец `family_status`:

In [17]:
df['family_status'].value_counts() # Просмотр уникальных значений семейного положения и подсчёт их количества 

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

Проверяем столбец `gender`:

In [18]:
df['gender'].value_counts() # Просмотр уникальных значений семейного положения и подсчёт их количества 

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

Посмотрим данные этого заёмщика c полом XNA

In [19]:
df[df['gender'] == 'XNA'] 

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10701,0,2358,24,неоконченное высшее,2,гражданский брак,1,XNA,компаньон,0,203905,покупка недвижимости


Остальные данные по клиенту с гендером XNA вполне валидны, следовательно есть смысл оставить данную строку, а гендер заменим на F, тк их примерно в 2 раза больше, чем M, а следовательно это в меньшей степени повлияет на исследование.

In [20]:
def xna(gender): # создадим функцию которой передаём корректные значения в столбце гендер и далее пропишем условия для каждого варианта возможного значения
    if gender == 'XNA':
        return 'F'
    if gender == 'F':
        return 'F'
    if gender == 'M':
        return 'M'
df['gender'] = df['gender'].apply(xna) # с помощью функции apply перепишим значения в столбце 'gender' используя функцию в которую прописали более подходящее значение гендера


Просмотр уникальных значений семейного положения и подсчёт их количества 

In [21]:
df['gender'].value_counts() 

F    14237
M     7288
Name: gender, dtype: int64

Проверяем столбец `income_type`:

In [22]:
df['income_type'].value_counts() # Просмотр уникальных значений типа занятости и подсчёт их количества 

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
безработный            2
предприниматель        2
студент                1
в декрете              1
Name: income_type, dtype: int64

Проверяем столбец `purpose`:

In [23]:
df['purpose'].value_counts() # Просмотр уникальных значений целей получения кредита и подсчёт их количества 

свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
покупка жилья для сдачи                   653
операции с жильем                         653
операции с коммерческой недвижимостью     651
покупка жилья                             647
жилье                                     647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
строительство недвижимости                620
покупка своего жилья                      620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

В столбцах `family_status`, `gender` и `income_type` дубликатов не обнаружено, а ошибочное значение из столбца `gender` мы удалили
<br>
Дубликаты есть в столбцах `education` и `purpose`.

Сначала приведём к нижнему регистру значения в стобцах `education`, `family_status` и `gender`, чтобы весь `DataFrame` был в нижнем регистре:

In [24]:
df['education'] = df['education'].str.lower() # Приводим к нижнему регистру столбец 'education'
df['family_status'] = df['family_status'].str.lower() # Приводим к нижнему регистру столбец 'family_status'
df['gender'] = df['gender'].str.lower() # Приводим к нижнему регистру столбец 'gender'

Проверим остались ли дубликаты в столбце `education`:

In [25]:
df['education'].value_counts() # Просмотр уникальных значений образования и подсчёт их количества

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

**Вывод**

Дубликаты из всех столбцов с типом `object` удалены, кроме столбца `purpose`, с данным столбцом мы будем работать в следующих пунктах.
<br>
<br>
Поиск дубликатов осуществлялся методом `value_counts()`, а для удаления дубликатов в столбце `education` было достаточно воспользоваться методом `str.lower()`, чтобы привести все значения в столбце к нижнему регистру.
<br>
<br>
Возможные причины появления дубликатов - это возможность в произвольной форме заполнять поля `education` и `purpose` со стороны потенциальных заёмщиков.

### Лемматизация

Для дальнейшего создания нового столбца с категорезированными данными из столбца `purpose` воспользуемся изначально лемматизацией, чтобы привести слова к их словарной форме и посмотреть какие слова встречаются чаще всего.
<br>
Воспользуемся библиотекой `pymystem3`:

In [26]:
%%time

m = Mystem()

str_porpose = ' '.join(df['purpose']) #Записываем все значения строк из столбца 'purpose' в одну строку через пробел
lemmas = m.lemmatize(str_porpose) #Лемматизируем строку со всеми значениями, которую получили на предыдущем шаге из стобца 'purpose'
Counter(lemmas) #Считаем число лемматизированные слов, чтобы выбрать наиболее встречающиеся причины

Wall time: 3.42 s


Counter({'покупка': 5912,
         ' ': 55201,
         'жилье': 4473,
         'приобретение': 462,
         'автомобиль': 4315,
         'дополнительный': 909,
         'образование': 4022,
         'сыграть': 774,
         'свадьба': 2348,
         'операция': 2610,
         'с': 2924,
         'на': 2233,
         'проведение': 777,
         'для': 1294,
         'семья': 641,
         'недвижимость': 6367,
         'коммерческий': 1315,
         'жилой': 1233,
         'строительство': 1881,
         'собственный': 635,
         'подержать': 858,
         'свой': 2235,
         'со': 630,
         'заниматься': 908,
         'сделка': 944,
         'получение': 1316,
         'высокий': 1375,
         'подержанный': 110,
         'профильный': 436,
         'сдача': 653,
         'ремонт': 612,
         '\n': 1})

**Вывод**

По результатам леммитизации будем в следующем пункте делать категоризацию данных по части слов из леммитизации`'недвиж', 'автомоб', 'образов', 'свадьб', 'ремонт'`

### Категоризация данных

Чтобы удалить дубликаты в столбце `purpose` воспользуемся категоризацией по наиболее встречающимся словам, которые мы нашли в предыдущем пункте.

In [27]:
def purpose_new(row):  # создадим функцию которой передаём значение наиболее часто встречающихся слов
    if 'автомоб' in row: # прописываем условие для покупки автомобиля
        return 'покупка автомобиля'
    elif 'ремонт' in row: # прописываем условие для оплаты ремонта
        return 'оплата ремонта'
    elif 'свадьб' in row: # прописываем условие для оплаты свадьбы
        return 'оплата свадьбы'
    elif 'образов' in row: # прописываем условие для оплаты образования
        return 'оптата образования'
    else: # прописываем условие для покупки недвижимости
        return 'покупка недвижимости'
    
df['purpose'] = df['purpose'].apply(purpose_new) # с помощью функции apply перепишим значения в столбце 'purpose' используя функцию в которую прописали категоризацию

df.head(10) # выведем для просмотра и дальнейшего исследования верхние 10 строк таблицы

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,f,сотрудник,0,253875,покупка недвижимости
1,1,4024,36,среднее,1,женат / замужем,0,f,сотрудник,0,112080,покупка автомобиля
2,0,5623,33,среднее,1,женат / замужем,0,m,сотрудник,0,145885,покупка недвижимости
3,3,4124,32,среднее,1,женат / замужем,0,m,сотрудник,0,267628,оптата образования
4,0,340266,53,среднее,1,гражданский брак,1,f,пенсионер,0,158616,оплата свадьбы
5,0,926,27,высшее,0,гражданский брак,1,m,компаньон,0,255763,покупка недвижимости
6,0,2879,43,высшее,0,женат / замужем,0,f,компаньон,0,240525,покупка недвижимости
7,0,152,50,среднее,1,женат / замужем,0,m,сотрудник,0,135823,оптата образования
8,2,6929,35,высшее,0,гражданский брак,1,f,сотрудник,0,95856,оплата свадьбы
9,0,2188,41,среднее,1,женат / замужем,0,m,сотрудник,0,144425,покупка недвижимости


**Вывод**

Данные приведены к единому стилю, удалены дубликаты, теперь можно приступить  к анализу данных и ответам на вопросы поставленные в описании проекта.

## Шаг 3. Ответьте на вопросы

### Есть ли зависимость между наличием детей и возвратом кредита в срок?

Изначально отфильтруем данные по наличию пропуска платежа и детям, чтобы изучить данные:

In [28]:
df.groupby('debt')['children'].value_counts()

debt  children
0     0           13086
      1            4420
      2            1861
      3             303
      20             68
      4              37
      5               9
1     0            1063
      1             445
      2             194
      3              27
      20              8
      4               4
Name: children, dtype: int64

В данных есть подозрительное число детей в размере 20, но так как в задании не ставится вопрос о зависимости числа детей, то нас интересуют заёмщики с детьми или без детей, число детей при этом значение не имеет.

In [29]:
def children(kid): # создадим функцию которой передаём корректные значения в столбце 'children' и далее пропишем условия для каждого варианта возможного значения
    if kid == 0:
        return 0
    if kid == 1:
        return 1
    if kid == 2:
        return 2
    if kid == 3:
        return 3
    if kid == 4:
        return 4
    if kid == 5:
        return 5
    if kid == 20:
        return 2        
df['children'] = df['children'].apply(children) # с помощью функции apply перепишим значения в столбце 'children' используя функцию в которую прописали более подходящее значения


Изначально отфильтруем данные по наличию пропуска платежа и детям с учётом исправленных данных по числу детей, чтобы изучить данные повторно:

In [30]:
df.groupby('debt')['children'].value_counts()

debt  children
0     0           13086
      1            4420
      2            1929
      3             303
      4              37
      5               9
1     0            1063
      1             445
      2             202
      3              27
      4               4
Name: children, dtype: int64

Найдем процентное отношение заёмщиков, которые пропустили платёж без детей:

In [31]:
no_children = df[df['children'] == 0]['debt'].sum() / df[df['children'] == 0]['debt'].count()
no_children

0.0751289843805216

Найдем процентное отношение заёмщиков, которые пропустили платёж и имеют детей:

In [32]:
have_children = df[df['children'] != 0]['debt'].sum() / df[df['children'] != 0]['debt'].count()
have_children

0.09191973969631237

Теперь посчитаем на сколько процентов чаще допускаю просрочку заёмщики с детьми:

In [33]:
(df[df['children'] != 0]['debt'].sum() / df[df['children'] != 0]['debt'].count()) / (df[df['children'] == 0]['debt'].sum() / df[df['children'] == 0]['debt'].count())

1.2234923771995518

**Вывод**

Из полученных значений можно сделать вывод, что наличие детей влияет на возврат кредита в срок, тк заёмщики без детей допускают просрочку в 7,51% случаев, а заёмщики с детьми в 9,19% случаем.
<br>
<br>
Получается, что заёмщики с детьми допускают просрочки на 22,35% чаще, чем заёмщики без детей.
<br>
<br>

### Есть ли зависимость между семейным положением и возвратом кредита в срок?

Изначально отфильтруем данные по наличию пропуска платежа и семейному положению, чтобы изучить данные:

In [34]:
df.groupby('debt')['family_status'].value_counts()

debt  family_status        
0     женат / замужем          11449
      гражданский брак          3789
      не женат / не замужем     2539
      в разводе                 1110
      вдовец / вдова             897
1     женат / замужем            931
      гражданский брак           388
      не женат / не замужем      274
      в разводе                   85
      вдовец / вдова              63
Name: family_status, dtype: int64

Найдем процентное отношение заёмщиков, которые пропустили платёж и при этом женаты/замужем:

In [35]:
debt_marriage = df[df['family_status'] == 'женат / замужем']['debt'].sum() / df[df['family_status'] == 'женат / замужем']['debt'].count()
debt_marriage

0.07520193861066236

Найдем процентное отношение заёмщиков, которые пропустили и при этом в гражданском браке:

In [36]:
debt_civil_marriage = df[df['family_status'] == 'гражданский брак']['debt'].sum() / df[df['family_status'] == 'гражданский брак']['debt'].count()
debt_civil_marriage

0.09288963370840315

Найдем процентное отношение заёмщиков, которые пропустили и при этом не женаты / не замужем:

In [37]:
debt_not_married = df[df['family_status'] == 'не женат / не замужем']['debt'].sum() / df[df['family_status'] == 'не женат / не замужем']['debt'].count()
debt_not_married

0.09740490579452542

Найдем процентное отношение заёмщиков, которые пропустили и при этом в разводе:

In [38]:
debt_divorce = df[df['family_status'] == 'в разводе']['debt'].sum() / df[df['family_status'] == 'в разводе']['debt'].count()
debt_divorce

0.07112970711297072

Найдем процентное отношение заёмщиков, которые пропустили и при этом являются вдовцом / вдовой:

In [39]:
debt_widowers = df[df['family_status'] == 'вдовец / вдова']['debt'].sum() / df[df['family_status'] == 'вдовец / вдова']['debt'].count()
debt_widowers

0.065625

**Вывод**

Из полученных значений можно сделать вывод, что семейное положение влияет на возврат кредита в срок. 
<br>
Процент просрочек при различных семейных положениях выглядит так:

In [40]:
def my_mean(x): return '{:.2%} '.format(x.mean())

df.groupby('family_status')['debt'].agg(['count', 'sum', my_mean])

,count,sum,my_mean
family_status,,,
в разводе,1195,85,7.11%
вдовец / вдова,960,63,6.56%
гражданский брак,4177,388,9.29%
женат / замужем,12380,931,7.52%
не женат / не замужем,2813,274,9.74%


В диапазон более надёжных заёмщиков попали люди, которые были в официальном браке ранее (вдовы/вдовцы и разведённые) и которые находятся сейчас в браке, диапазон значений просрочки у них от 6,56% до 7,52%.
<br>
<br>
Наименее надёжными заёмщиками являются люди, которые никогда не были в официальном браке (гражданский брак и не женат/не замужем), диапазон значений просрочки у них значительно выше от 9,29% до 9,74%.
<br>
<br>
Получается, что есть прямая зависимость семейного положения клиента и процента просрочек. Люди, которые были в официальном браке являются значительно более ответственными и как следствие более надёжными заёмщиками.
<br>
<br>
Сгруппируем всех заёмщиков в 2 категории, кто был в официальном браке и кто никогда не был, а далее найдём среднее значение группы и посчитаем на сколько процентов больше допускают просрочку люди, которые никогда не были в официальном браке:

In [41]:
((debt_civil_marriage + debt_not_married)/2) / ((debt_marriage + debt_divorce + debt_widowers)/3)

1.3466990302657267

Получается, что в среднем заёмщики, которые не был никогда в официальном браке допускают просрочку на 34,67% чаще, чем заёмщики которые находятся или были ранее в официальном браке. 

###  Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Изначально отфильтруем данные по наличию пропуска платежа и ежемесячному доходу, чтобы изучить данные:

In [42]:
df.groupby('debt')['total_income'].value_counts()

debt  total_income
0     142594          1007
      172357           479
      118514           379
      150447           140
      92764              3
                      ... 
1     711866             1
      822426             1
      866784             1
      1030899            1
      2200852            1
Name: total_income, Length: 18726, dtype: int64

Есть несколько больших групп с одинаковым доходом (118514, 142594, 150447, 172357), попробуем на них проверить гипотезу касательно зависимости возврата кредита в срок от уровня дохода

In [43]:
df[df['total_income'] == 118514]['debt'].sum() / df[df['total_income'] == 118514]['debt'].count()

0.08454106280193237

In [44]:
df[df['total_income'] == 142594]['debt'].sum() / df[df['total_income'] == 142594]['debt'].count()

0.08868778280542987

In [45]:
df[df['total_income'] == 150447]['debt'].sum() / df[df['total_income'] == 150447]['debt'].count()

0.047619047619047616

In [46]:
df[df['total_income'] == 172357]['debt'].sum() / df[df['total_income'] == 172357]['debt'].count()

0.05893909626719057

Кроме этого возьмём для расчёта минимальное, медианное и максимальное значение столбца и для данных значений тоже проверим гипотезу:

In [47]:
df['total_income'].min()

20667

In [48]:
df[df['total_income'] == 20667]['debt'].sum() / df[df['total_income'] == 20667]['debt'].count()

1.0

In [49]:
df['total_income'].median()

142594.0

In [50]:
df[df['total_income'] == 142594]['debt'].sum() / df[df['total_income'] == 142594]['debt'].count()

0.08868778280542987

In [51]:
df['total_income'].max()

2265604

In [52]:
df[df['total_income'] == 2265604]['debt'].sum() / df[df['total_income'] == 2265604]['debt'].count()

0.0

Разделим доход всех заёмщиков на 4 категорий. Возьмём медианное значение всего списка, а затем ещё раз применим медианное значение к двум полученным диапазонам:

In [53]:
category_2 = df['total_income'].median() # медианное значение всего списка
category_2

142594.0

In [54]:
df_median_1 = df[df['total_income'] <= category_2]
category_1 = df_median_1['total_income'].median() # медианное значение первой половины списка
category_1

108301.0

In [55]:
df_median_2 = df[df['total_income'] >= category_2]
category_3 = df_median_2['total_income'].median() # медианное значение второй половины списка
category_3

188383.0

Считаем процент  невозврата для первой категории от минимального значения дохода до медианного значения первой половины списка:

In [56]:
percent_1 = df[df['total_income'] <= category_1]['debt'].sum() / df[df['total_income'] <= category_1]['debt'].count() # процент невозврата первой категории
percent_1

0.07995598752980011

Считаем процент  невозврата для второй категории от медианного значения первой половины списка дохода до медианного значения всего списка:

In [57]:
percent_2 = df[(df['total_income'] >= category_1) & (df['total_income'] <= category_2)]['debt'].sum() / df[(df['total_income'] >= category_1) & (df['total_income'] <= category_2)]['debt'].count()# процент невозврата второй категории
percent_2

0.08692462864478269

Считаем процент  невозврата для третьей категории от медианного значения всего списка до медианного значения второй половины :

In [58]:
percent_3 = df[(df['total_income'] >= category_2) & (df['total_income'] <= category_3)]['debt'].sum() / df[(df['total_income'] >= category_2) & (df['total_income'] <= category_3)]['debt'].count()# процент невозврата третьей категории
percent_3

0.08459832850076753

Считаем процент невозврата для четвёртой категории от медианного значения второй половины списка до максимального значения дохода:

In [59]:
percent_4 = df[df['total_income'] >= category_3]['debt'].sum() / df[df['total_income'] >= category_3]['debt'].count()# процент невозврата третьей категории
percent_4

0.07385297629200069

Сведём полученные значения просрочек для  каждого диапазона дохода в таблицу:

In [60]:
columns = ['total_income_min', 'total_income_max', 'delay']
data = [[int(df['total_income'].min()), category_1, percent_1], 
        [category_1, category_2, percent_2], 
        [category_2, category_3, percent_3], 
        [category_3, int(df['total_income'].max()), percent_4]]

display(pd.DataFrame(data=data, columns=columns))

,total_income_min,total_income_max,delay
0,20667.0,108301.0,0.079956
1,108301.0,142594.0,0.086925
2,142594.0,188383.0,0.084598
3,188383.0,2265604.0,0.073853


**Вывод**

На основании полученых данных можно сделать вывод, что доход не слишком значительно и не прямо пропорционально влияет на возврат кредита в срок. 
<br>
<br>
Наименьший процент просрочки - 7,39% у клиентов с наибольшим доходом, при этом наибольший процент просрочки - 8,69% у клиентов второй категории, то есть не у клиентов с наименьшим доходом. 
<br>
<br>
И в целом получившиеся значения просрочки с разбивкой по категориям дохода имеет гораздо меньший разбег значений, чем в предыдущих исследованиях.

### Как разные цели кредита влияют на его возврат в срок?

Изначально отфильтруем данные по наличию пропуска платежа и цели кредита, чтобы изучить данные:

In [61]:
df.groupby('purpose')['debt'].value_counts()

purpose               debt
оплата ремонта        0        577
                      1         35
оплата свадьбы        0       2162
                      1        186
оптата образования    0       3652
                      1        370
покупка автомобиля    0       3912
                      1        403
покупка недвижимости  0       9481
                      1        747
Name: debt, dtype: int64

Найдем процентное отношение заёмщиков, которые пропустили платёж с целью кредита "покупка недвижимости":

In [62]:
df[df['purpose'] == 'покупка недвижимости']['debt'].sum() / df[df['purpose'] == 'покупка недвижимости']['debt'].count()

0.07303480641376613

Найдем процентное отношение заёмщиков, которые пропустили платёж с целью кредита "покупка автомобиля":

In [63]:
car = df[df['purpose'] == 'покупка автомобиля']['debt'].sum() / df[df['purpose'] == 'покупка автомобиля']['debt'].count()
car

0.09339513325608342

Найдем процентное отношение заёмщиков, которые пропустили платёж с целью кредита "оптата образования":

In [64]:
df[df['purpose'] == 'оптата образования']['debt'].sum() / df[df['purpose'] == 'оптата образования']['debt'].count()


0.0919940328194928

Найдем процентное отношение заёмщиков, которые пропустили платёж с целью кредита "оплата свадьбы":

In [65]:
df[df['purpose'] == 'оплата свадьбы']['debt'].sum() / df[df['purpose'] == 'оплата свадьбы']['debt'].count()

0.07921635434412266

Найдем процентное отношение заёмщиков, которые пропустили платёж с целью кредита "оплата ремонта":

In [66]:
repair = df[df['purpose'] == 'оплата ремонта']['debt'].sum() / df[df['purpose'] == 'оплата ремонта']['debt'].count()
repair

0.05718954248366013

В столбце `debt` единицами обозначено наличие задолженности. Нулем — ее отсутствие. Так, сумма столбца будет равна сумме единичек или числу должников, а применение метода `count` вернет общее количество кредитов. Таким образом, для поиска доли должников надо разделить сумму на количество, а это не что иное, как среднее. Можно это сделать в одной группировке:

In [67]:
def my_mean(x): return '{:.2%} '.format(x.mean()) # зменяем среднее значение для каждой категории функцией, чтобы привести значение к виду процентов

df.groupby('purpose')['debt'].agg(['count', 'sum', my_mean])

,count,sum,my_mean
purpose,,,
оплата ремонта,612,35,5.72%
оплата свадьбы,2348,186,7.92%
оптата образования,4022,370,9.20%
покупка автомобиля,4315,403,9.34%
покупка недвижимости,10228,747,7.30%


**Вывод**

Из полученных значений можно сделать вывод, что цель кредита влияет на возврат кредита в срок. 
<br>
Процент просрочек при различных целях кредита выглядит так:
* оплата ремонта - 5,72%
* покупка недвижимости - 7,3%
* оплата свадьбы - 7,92%
* оптата образования - 9,2%
* покупка автомобиля - 9,34%
<br>
Получается, что процент просрочек в кредитах, которые связаны с недвижимостью (оплата ремонта или покупка недвижимости) значительно более низкий, чем в кредитах на движимое имущество или оплату услуг.

## Шаг 4. Общий вывод

Мы проверили 4 гипотезы и установили:
* Наличие детей негативно влияет на возврат кредита в срок. Получается, что есть прямая зависимость от ниличия детей у клиента и процента просрочек.
* Семейное положение влияет на возврат кредита в срок, меньше всего просрочек у вдовцов/вдов, а больше всего у не женатых/не замужных. Получается, что есть прямая зависимость семейного положения клиента и процента просрочек. Люди, которые были в официальном браке являются значительно более ответственными и как следствие более надёжными заёмщиками.
* Доход заёмщика не слишком значительно и не прямо пропорционально влияет на возврат кредита в срок. 
* Цель кредита влияет на возврат кредита в срок, меньше всего просрочек у тех кто оплачивал ремонт, а больше всего у тех кто покупал автомобиль 

В таблице ниже предствалены данные с категорией минимального и максимального процента просрочки по трём гипотезам, где подтвердилась зависимость просрочки от соответствующей категории выборки:

In [68]:
columns_conclusion = ['hypothesis', 'category_min','delay_min_delay', 'category_min_delay', 'delay_max']
data_conclusion = [['влияние детей на возврат кредита', 'нет детей', no_children, 'есть дети', have_children], 
        ['влияние семейного положения на возвратом кредита', 'вдовец / вдова', debt_widowers, 'не женат / не замужем', debt_not_married], 
        ['влияние цели кредита на его возврат', 'оплата ремонта', repair, 'покупка автомобиля', car]]

display(pd.DataFrame(data=data_conclusion, columns=columns_conclusion))

,hypothesis,category_min,delay_min_delay,category_min_delay,delay_max
0,влияние детей на возврат кредита,нет детей,0.075129,есть дети,0.091920
1,влияние семейного положения на возвратом кредита,вдовец / вдова,0.065625,не женат / не замужем,0.097405
2,влияние цели кредита на его возврат,оплата ремонта,0.057190,покупка автомобиля,0.093395
